In [1]:
%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 796.8 kB/s eta 0:52:31
     ---------------------------------------- 0.0/2.5 GB 796.8 kB/s eta 0:52:31
     ---------------------------------------- 0.0/2.5 GB 744.7 kB/s eta 0:56:11
     ---------------------------------------- 0.0/2.5 GB 824.4 kB/s eta 0:50:45
     ---------------------------------------- 0.0/2.5 GB 849.0 kB/s eta 0:49:16
     ---------------------------------------- 0.0/2.5 GB 855.6 kB/s eta 0:48:53
     ---------------------------------------- 0.0/2.5 GB 898.8 kB/s eta 0:46:32
     ---------------------------------------- 0.0/2.5 GB 898.8 kB/s eta 0:46:32
     ---------------------------------------- 0.0/2.5 GB 924.4 kB/s eta 0:45:14
     ---

In [2]:
import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric

Installing packages for torch-2.5.1+cu124...
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
     ------------ --------------------------- 0.5/1.7 MB 2.1 MB/s eta 0:00:01
     ------------------ --------------------- 0.8/1.7 MB 1.1 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.5 MB 2.8 MB/s eta 0:00:02
     ------------ --------------------------- 1.0/3.5 MB 2.5 MB/s eta 0:00:01
     ------------------ --------------------- 1.6/3.5 MB 2.5 MB/s eta 0:00:01
     --------------------- ------------------ 1.8/3.5 MB 2.6 MB/s eta 0:00:01
     --------------------------- ------------ 2.4/3.

In [4]:
%pip install scikit-learn imbalanced-learn xgboost

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.1 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.2 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.1 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.1 MB 1.1 MB/s eta 0:00:09
   -------- ----------------

In [6]:
%pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 1.3 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 1.6 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Hyperparameter Optimization Citeseer
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.05, 1.0),
        "K": trial.suggest_int("K", 17, 23),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0, 0.5),
        "margin": trial.suggest_float("margin", 0.05, 0.99),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 0.99),
        "ratio": trial.suggest_float("ratio", 0, 0.5),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=250)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-23 16:39:28,604] A new study created in memory with name: no-name-9ffa1d7c-26c0-45c7-8f28-cda45272791a


Running experiment with seed=1:
 - alpha=0.11649134267148638, K=23, layers=2, hidden=64, out=64
 - norm=layernorm, dropout=0.4461186569349865, margin=0.3596784126857589, lpl_weight=0.4917957802880062
 - ratio=0.41429964058629537, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.7758, LPL: 1.3863, Contrastive: 0.1851
 - Metrics: Accuracy=0.8662, F1=0.7362, Recall=0.8859, Precision=0.6298
Running experiment with seed=2:
 - alpha=0.11649134267148638, K=23, layers=2, hidden=64, out=64
 - norm=layernorm, dropout=0.4461186569349865, margin=0.3596784126857589, lpl_weight=0.4917957802880062
 - ratio=0.41429964058629537, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.7758, LPL: 1.3863, Contrastive: 0.1851
 - Metrics: Accuracy=0.8623, F1=0.7365, Recall=0.9130, Precision=0.6172
Running experiment with seed=3:
 - alpha=0.11649134267148638, K=23, layers=2, hidden=64, out=64
 - norm=layernorm, dropout=0.4461186569349865, margin=0.3596784126857589, lpl_weight=0.491

[I 2025-02-23 16:40:10,326] Trial 0 finished with value: 0.742725162437942 and parameters: {'alpha': 0.11649134267148638, 'K': 23, 'layers': 2, 'hidden_channels': 64, 'out_channels': 64, 'norm': 'layernorm', 'dropout': 0.4461186569349865, 'margin': 0.3596784126857589, 'lpl_weight': 0.4917957802880062, 'ratio': 0.41429964058629537, 'aggregation': 'sum'}. Best is trial 0 with value: 0.742725162437942.


 - Metrics: Accuracy=0.8629, F1=0.7333, Recall=0.8944, Precision=0.6214
Done. Results written to citeseer_experimentations\citeseer_scar_2302163928.csv.
Average F1 over 5 seeds: 0.7427 ± 0.0121
Running experiment with seed=1:
 - alpha=0.6059255158964738, K=19, layers=1, hidden=64, out=64
 - norm=graphnorm, dropout=0.3640950535712533, margin=0.5272515879863956, lpl_weight=0.6949631590035429
 - ratio=0.12104728679579396, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0125, LPL: 1.3863, Contrastive: 0.1608
Epoch 50, Loss: 0.9816, LPL: 1.3863, Contrastive: 0.0596
Epoch 100, Loss: 0.9812, LPL: 1.3863, Contrastive: 0.0584
Epoch 150, Loss: 0.9811, LPL: 1.3863, Contrastive: 0.0578
Epoch 200, Loss: 0.9810, LPL: 1.3863, Contrastive: 0.0575
Epoch 250, Loss: 0.9809, LPL: 1.3863, Contrastive: 0.0574
Epoch 300, Loss: 0.9809, LPL: 1.3863, Contrastive: 0.0573
 - Metrics: Accuracy=0.9213, F1=0.7930, Recall=0.7161, Precision=0.8885
Running experiment with seed=2:
 - alpha=0.6059255158

[I 2025-02-23 16:42:29,000] Trial 1 finished with value: 0.7882780224345753 and parameters: {'alpha': 0.6059255158964738, 'K': 19, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'norm': 'graphnorm', 'dropout': 0.3640950535712533, 'margin': 0.5272515879863956, 'lpl_weight': 0.6949631590035429, 'ratio': 0.12104728679579396, 'aggregation': 'sum'}. Best is trial 1 with value: 0.7882780224345753.


 - Metrics: Accuracy=0.9176, F1=0.7879, Recall=0.7261, Precision=0.8613
Done. Results written to citeseer_experimentations\citeseer_scar_2302164010.csv.
Average F1 over 5 seeds: 0.7883 ± 0.0091
Running experiment with seed=1:
 - alpha=0.08414660377308, K=21, layers=1, hidden=256, out=64
 - norm=layernorm, dropout=0.4300082654309305, margin=0.18332355498619252, lpl_weight=0.26142643342241484
 - ratio=0.25404025884572035, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.5211, LPL: 1.3863, Contrastive: 0.2149
Epoch 50, Loss: 0.4905, LPL: 1.3863, Contrastive: 0.1734
Epoch 100, Loss: 0.4884, LPL: 1.3863, Contrastive: 0.1706
 - Metrics: Accuracy=0.9152, F1=0.8115, Recall=0.8659, Precision=0.7635
Running experiment with seed=2:
 - alpha=0.08414660377308, K=21, layers=1, hidden=256, out=64
 - norm=layernorm, dropout=0.4300082654309305, margin=0.18332355498619252, lpl_weight=0.26142643342241484
 - ratio=0.25404025884572035, pos_weight=1, aggregation=sum, treatment=removal
Epoch

[I 2025-02-23 16:44:18,853] Trial 2 finished with value: 0.816100525732694 and parameters: {'alpha': 0.08414660377308, 'K': 21, 'layers': 1, 'hidden_channels': 256, 'out_channels': 64, 'norm': 'layernorm', 'dropout': 0.4300082654309305, 'margin': 0.18332355498619252, 'lpl_weight': 0.26142643342241484, 'ratio': 0.25404025884572035, 'aggregation': 'sum'}. Best is trial 2 with value: 0.816100525732694.


 - Metrics: Accuracy=0.9107, F1=0.8034, Recall=0.8659, Precision=0.7494
Done. Results written to citeseer_experimentations\citeseer_scar_2302164229.csv.
Average F1 over 5 seeds: 0.8161 ± 0.0139
Running experiment with seed=1:
 - alpha=0.5668746728173657, K=19, layers=2, hidden=256, out=128
 - norm=graphnorm, dropout=0.1765151743804142, margin=0.9721638666906344, lpl_weight=0.6500496331098355
 - ratio=0.34447683195700557, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.9229, LPL: 1.3863, Contrastive: 0.0622
 - Metrics: Accuracy=0.9125, F1=0.7932, Recall=0.7960, Precision=0.7904
Running experiment with seed=2:
 - alpha=0.5668746728173657, K=19, layers=2, hidden=256, out=128
 - norm=graphnorm, dropout=0.1765151743804142, margin=0.9721638666906344, lpl_weight=0.6500496331098355
 - ratio=0.34447683195700557, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.9229, LPL: 1.3863, Contrastive: 0.0622


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 15, 25),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=250)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 11:03:38,552] A new study created in memory with name: no-name-51768b11-9e2d-4414-b07e-f364f0127f2d


Running experiment with seed=1:
 - alpha=0.19318242593081886, K=15, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.42110600860832226, margin=0.9239927738665273, lpl_weight=0.9572821553708265
 - ratio=0.43303873683086513, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.3362, LPL: 1.3863, Contrastive: 0.2137
 - Metrics: Accuracy=0.8383, F1=0.6691, Recall=0.7760, Precision=0.5881
Running experiment with seed=2:
 - alpha=0.19318242593081886, K=15, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.42110600860832226, margin=0.9239927738665273, lpl_weight=0.9572821553708265
 - ratio=0.43303873683086513, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.3362, LPL: 1.3863, Contrastive: 0.2137
 - Metrics: Accuracy=0.8644, F1=0.7111, Recall=0.7917, Precision=0.6453
Running experiment with seed=3:
 - alpha=0.19318242593081886, K=15, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.42110600860832226, margin=0.9239927738665273, lpl_w

[I 2025-02-21 11:04:27,032] Trial 0 finished with value: 0.7005643791833988 and parameters: {'alpha': 0.19318242593081886, 'K': 15, 'layers': 3, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'graphnorm', 'dropout': 0.42110600860832226, 'margin': 0.9239927738665273, 'lpl_weight': 0.9572821553708265, 'ratio': 0.43303873683086513, 'aggregation': 'mean', 'treatment': 'removal'}. Best is trial 0 with value: 0.7005643791833988.


 - Metrics: Accuracy=0.8605, F1=0.7048, Recall=0.7903, Precision=0.6361
Done. Results written to citeseer_experimentations\citeseer_sar_2102110338.csv.
Average F1 over 5 seeds: 0.7006 ± 0.0159
Running experiment with seed=1:
 - alpha=0.8736943955842099, K=22, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.1369219263148594, margin=0.5533481495913736, lpl_weight=0.5616376297689698
 - ratio=0.3416643268199105, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.8284, LPL: 1.3863, Contrastive: 0.1137
Epoch 50, Loss: 0.8014, LPL: 1.3863, Contrastive: 0.0520
Epoch 100, Loss: 0.8010, LPL: 1.3863, Contrastive: 0.0511
Epoch 150, Loss: 0.8009, LPL: 1.3863, Contrastive: 0.0510
 - Metrics: Accuracy=0.7680, F1=0.6267, Recall=0.9244, Precision=0.4740
Running experiment with seed=2:
 - alpha=0.8736943955842099, K=22, layers=3, hidden=256, out=128
 - norm=graphnorm, dropout=0.1369219263148594, margin=0.5533481495913736, lpl_weight=0.5616376297689698
 - ratio=0.34166432681

[I 2025-02-21 11:06:36,575] Trial 1 finished with value: 0.62321083172147 and parameters: {'alpha': 0.8736943955842099, 'K': 22, 'layers': 3, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'graphnorm', 'dropout': 0.1369219263148594, 'margin': 0.5533481495913736, 'lpl_weight': 0.5616376297689698, 'ratio': 0.3416643268199105, 'aggregation': 'mean', 'treatment': 'relabeling'}. Best is trial 0 with value: 0.7005643791833988.


...

 - Metrics: Accuracy=0.9375, F1=0.8429, Recall=0.7960, Precision=0.8957
Done. Results written to citeseer_experimentations\citeseer_sar_2102122100.csv.
Average F1 over 5 seeds: 0.8314 ± 0.0085
Running experiment with seed=1:
 - alpha=0.46143822782565064, K=17, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.26755087144223877, margin=0.6422637921530638, lpl_weight=0.6800360078051488
 - ratio=0.11950420241251236, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.9998, LPL: 1.3863, Contrastive: 0.1784
 - Metrics: Accuracy=0.9261, F1=0.8139, Recall=0.7675, Precision=0.8663
Running experiment with seed=2:
 - alpha=0.46143822782565064, K=17, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.26755087144223877, margin=0.6422637921530638, lpl_weight=0.6800360078051488
 - ratio=0.11950420241251236, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.9998, LPL: 1.3863, Contrastive: 0.1784
 - Metrics: Accuracy=0.9291, F1=0.8201, Recall=0.7675, Prec

[I 2025-02-21 12:22:22,741] Trial 48 finished with value: 0.8243460344734677 and parameters: {'alpha': 0.46143822782565064, 'K': 17, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'layernorm', 'dropout': 0.26755087144223877, 'margin': 0.6422637921530638, 'lpl_weight': 0.6800360078051488, 'ratio': 0.11950420241251236, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 38 with value: 0.8455717941380525.


 - Metrics: Accuracy=0.9354, F1=0.8355, Recall=0.7789, Precision=0.9010
Done. Results written to citeseer_experimentations\citeseer_sar_2102122141.csv.
Average F1 over 5 seeds: 0.8243 ± 0.0073
Running experiment with seed=1:
 - alpha=0.5737913294462824, K=18, layers=3, hidden=256, out=128
 - norm=layernorm, dropout=0.2658876115886441, margin=0.7344059696913318, lpl_weight=0.7186557437315787
 - ratio=0.1466624311788599, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0325, LPL: 1.3863, Contrastive: 0.1288
 - Metrics: Accuracy=0.9068, F1=0.7693, Recall=0.7375, Precision=0.8040
Running experiment with seed=2:
 - alpha=0.5737913294462824, K=18, layers=3, hidden=256, out=128
 - norm=layernorm, dropout=0.2658876115886441, margin=0.7344059696913318, lpl_weight=0.7186557437315787
 - ratio=0.1466624311788599, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0325, LPL: 1.3863, Contrastive: 0.1288
 - Metrics: Accuracy=0.9092, F1=0.7726, Recall=0.7318, Precision=

[I 2025-02-21 12:23:05,622] Trial 49 finished with value: 0.7756226033086795 and parameters: {'alpha': 0.5737913294462824, 'K': 18, 'layers': 3, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'layernorm', 'dropout': 0.2658876115886441, 'margin': 0.7344059696913318, 'lpl_weight': 0.7186557437315787, 'ratio': 0.1466624311788599, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 38 with value: 0.8455717941380525.


 - Metrics: Accuracy=0.9137, F1=0.7817, Recall=0.7332, Precision=0.8371
Done. Results written to citeseer_experimentations\citeseer_sar_2102122222.csv.
Average F1 over 5 seeds: 0.7756 ± 0.0110
Best trial:
  Average F1: 0.8455717941380525
  Best parameters:
    alpha: 0.37723710900430574
    K: 20
    layers: 2
    hidden_channels: 256
    out_channels: 128
    norm: layernorm
    dropout: 0.2653392372389523
    margin: 0.5902726074174123
    lpl_weight: 0.5979423122947367
    ratio: 0.13594889546650762
    aggregation: sum
    treatment: removal


### Hyperparameter Optimization Cora
#### SCAR 

In [7]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 12:23:05,650] A new study created in memory with name: no-name-4468d66a-049b-414c-8ec5-8a1409d48901
Processing...
Done!


Running experiment with seed=1:
 - alpha=0.6590275255796161, K=34, layers=2, hidden=256, out=64
 - norm=graphnorm, dropout=0.252224978557246, margin=0.801479736149222, lpl_weight=0.10820398144332889
 - ratio=0.3781283040605532, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.2991, LPL: 1.3863, Contrastive: 0.1672
 - Metrics: Accuracy=0.7947, F1=0.7347, Recall=0.9413, Precision=0.6025
Running experiment with seed=2:
 - alpha=0.6590275255796161, K=34, layers=2, hidden=256, out=64
 - norm=graphnorm, dropout=0.252224978557246, margin=0.801479736149222, lpl_weight=0.10820398144332889
 - ratio=0.3781283040605532, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.2991, LPL: 1.3863, Contrastive: 0.1672
 - Metrics: Accuracy=0.7969, F1=0.7376, Recall=0.9450, Precision=0.6049
Running experiment with seed=3:
 - alpha=0.6590275255796161, K=34, layers=2, hidden=256, out=64
 - norm=graphnorm, dropout=0.252224978557246, margin=0.801479736149222, lpl_weight=0.108

[I 2025-02-21 12:23:41,175] Trial 0 finished with value: 0.7356870229007634 and parameters: {'alpha': 0.6590275255796161, 'K': 34, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'norm': 'graphnorm', 'dropout': 0.252224978557246, 'margin': 0.801479736149222, 'lpl_weight': 0.10820398144332889, 'ratio': 0.3781283040605532, 'aggregation': 'sum', 'treatment': 'relabeling'}. Best is trial 0 with value: 0.7356870229007634.


 - Metrics: Accuracy=0.7954, F1=0.7357, Recall=0.9425, Precision=0.6033
Done. Results written to cora_experimentations\cora_scar_2102122305.csv.
Average F1 over 5 seeds: 0.7357 ± 0.0029
Running experiment with seed=1:
 - alpha=0.9541489676457723, K=33, layers=3, hidden=128, out=256
 - norm=graphnorm, dropout=0.4793875130120685, margin=0.3521159342119853, lpl_weight=0.9293264066487583
 - ratio=0.23356207667182172, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.3006, LPL: 1.3863, Contrastive: 0.1732
Epoch 50, Loss: 1.2961, LPL: 1.3863, Contrastive: 0.1096
Epoch 100, Loss: 1.2960, LPL: 1.3863, Contrastive: 0.1085
 - Metrics: Accuracy=0.9298, F1=0.8872, Recall=0.9132, Precision=0.8626
Running experiment with seed=2:
 - alpha=0.9541489676457723, K=33, layers=3, hidden=128, out=256
 - norm=graphnorm, dropout=0.4793875130120685, margin=0.3521159342119853, lpl_weight=0.9293264066487583
 - ratio=0.23356207667182172, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Lo

[I 2025-02-21 12:24:50,664] Trial 1 finished with value: 0.8753877722852119 and parameters: {'alpha': 0.9541489676457723, 'K': 33, 'layers': 3, 'hidden_channels': 128, 'out_channels': 256, 'norm': 'graphnorm', 'dropout': 0.4793875130120685, 'margin': 0.3521159342119853, 'lpl_weight': 0.9293264066487583, 'ratio': 0.23356207667182172, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 1 with value: 0.8753877722852119.


...

 - Metrics: Accuracy=0.9365, F1=0.8943, Recall=0.8900, Precision=0.8988
Done. Results written to cora_experimentations\cora_scar_2102135453.csv.
Average F1 over 5 seeds: 0.8861 ± 0.0066
Running experiment with seed=1:
 - alpha=0.12365278694092605, K=34, layers=1, hidden=256, out=128
 - norm=layernorm, dropout=0.46984909590355645, margin=0.3669963811065297, lpl_weight=0.19629883858215663
 - ratio=0.16881198834078698, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.4163, LPL: 1.3863, Contrastive: 0.1793
 - Metrics: Accuracy=0.9280, F1=0.8793, Recall=0.8680, Precision=0.8908
Running experiment with seed=2:
 - alpha=0.12365278694092605, K=34, layers=1, hidden=256, out=128
 - norm=layernorm, dropout=0.46984909590355645, margin=0.3669963811065297, lpl_weight=0.19629883858215663
 - ratio=0.16881198834078698, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.4163, LPL: 1.3863, Contrastive: 0.1793
 - Metrics: Accuracy=0.9309, F1=0.8842, Recall=0.8729, Pre

[I 2025-02-21 13:55:53,265] Trial 98 finished with value: 0.8850646625310269 and parameters: {'alpha': 0.12365278694092605, 'K': 34, 'layers': 1, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'layernorm', 'dropout': 0.46984909590355645, 'margin': 0.3669963811065297, 'lpl_weight': 0.19629883858215663, 'ratio': 0.16881198834078698, 'aggregation': 'sum', 'treatment': 'relabeling'}. Best is trial 17 with value: 0.8982590092902513.


 - Metrics: Accuracy=0.9387, F1=0.8971, Recall=0.8851, Precision=0.9095
Done. Results written to cora_experimentations\cora_scar_2102135523.csv.
Average F1 over 5 seeds: 0.8851 ± 0.0072
Running experiment with seed=1:
 - alpha=0.6435002303643785, K=32, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.443769204599643, margin=0.46092482861682205, lpl_weight=0.9867928890424483
 - ratio=0.22919340802586002, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.3702, LPL: 1.3863, Contrastive: 0.1677
 - Metrics: Accuracy=0.9321, F1=0.8882, Recall=0.8936, Precision=0.8829
Running experiment with seed=2:
 - alpha=0.6435002303643785, K=32, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.443769204599643, margin=0.46092482861682205, lpl_weight=0.9867928890424483
 - ratio=0.22919340802586002, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.3702, LPL: 1.3863, Contrastive: 0.1677
 - Metrics: Accuracy=0.9313, F1=0.8880, Recall=0.9010, Precision=0.875

[I 2025-02-21 13:56:22,488] Trial 99 finished with value: 0.8889667335752852 and parameters: {'alpha': 0.6435002303643785, 'K': 32, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'norm': 'layernorm', 'dropout': 0.443769204599643, 'margin': 0.46092482861682205, 'lpl_weight': 0.9867928890424483, 'ratio': 0.22919340802586002, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 17 with value: 0.8982590092902513.


 - Metrics: Accuracy=0.9361, F1=0.8961, Recall=0.9120, Precision=0.8808
Done. Results written to cora_experimentations\cora_scar_2102135553.csv.
Average F1 over 5 seeds: 0.8890 ± 0.0045
Best trial:
  Average F1: 0.8982590092902513
  Best parameters:
    alpha: 0.26625526948518147
    K: 30
    layers: 1
    hidden_channels: 256
    out_channels: 256
    norm: layernorm
    dropout: 0.30136742497786895
    margin: 0.5111785498137569
    lpl_weight: 0.7262936518886128
    ratio: 0.23112465661992612
    aggregation: sum
    treatment: removal


### Hyperparameter Optimization Cora
#### SAR 

In [8]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 13:56:22,520] A new study created in memory with name: no-name-752611fe-0e4a-499e-ab0b-538cb1576b4a


Running experiment with seed=1:
 - alpha=0.4644361873267727, K=29, layers=2, hidden=256, out=256
 - norm=graphnorm, dropout=0.21451522129748124, margin=0.5774351883518448, lpl_weight=0.6894272980504211
 - ratio=0.4082216469176174, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.9811, LPL: 1.3863, Contrastive: 0.0815
 - Metrics: Accuracy=0.8852, F1=0.8220, Recall=0.8778, Precision=0.7729
Running experiment with seed=2:
 - alpha=0.4644361873267727, K=29, layers=2, hidden=256, out=256
 - norm=graphnorm, dropout=0.21451522129748124, margin=0.5774351883518448, lpl_weight=0.6894272980504211
 - ratio=0.4082216469176174, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.9811, LPL: 1.3863, Contrastive: 0.0815
 - Metrics: Accuracy=0.8925, F1=0.8329, Recall=0.8863, Precision=0.7855
Running experiment with seed=3:
 - alpha=0.4644361873267727, K=29, layers=2, hidden=256, out=256
 - norm=graphnorm, dropout=0.21451522129748124, margin=0.5774351883518448, lpl_weight

[I 2025-02-21 13:56:56,438] Trial 0 finished with value: 0.8202565278191223 and parameters: {'alpha': 0.4644361873267727, 'K': 29, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'norm': 'graphnorm', 'dropout': 0.21451522129748124, 'margin': 0.5774351883518448, 'lpl_weight': 0.6894272980504211, 'ratio': 0.4082216469176174, 'aggregation': 'mean', 'treatment': 'removal'}. Best is trial 0 with value: 0.8202565278191223.


 - Metrics: Accuracy=0.8796, F1=0.8152, Recall=0.8790, Precision=0.7600
Done. Results written to cora_experimentations\cora_sar_2102135622.csv.
Average F1 over 5 seeds: 0.8203 ± 0.0069
Running experiment with seed=1:
 - alpha=0.8012605305582338, K=25, layers=2, hidden=64, out=256
 - norm=layernorm, dropout=0.15720674171130133, margin=0.3105436717700593, lpl_weight=0.5391286318590843
 - ratio=0.11221640474839122, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.8263, LPL: 1.3863, Contrastive: 0.1712
Epoch 50, Loss: 0.8043, LPL: 1.3863, Contrastive: 0.1234
Epoch 100, Loss: 0.8037, LPL: 1.3863, Contrastive: 0.1221
 - Metrics: Accuracy=0.9129, F1=0.8386, Recall=0.7494, Precision=0.9519
Running experiment with seed=2:
 - alpha=0.8012605305582338, K=25, layers=2, hidden=64, out=256
 - norm=layernorm, dropout=0.15720674171130133, margin=0.3105436717700593, lpl_weight=0.5391286318590843
 - ratio=0.11221640474839122, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Los

[I 2025-02-21 13:57:49,400] Trial 1 finished with value: 0.835938973528496 and parameters: {'alpha': 0.8012605305582338, 'K': 25, 'layers': 2, 'hidden_channels': 64, 'out_channels': 256, 'norm': 'layernorm', 'dropout': 0.15720674171130133, 'margin': 0.3105436717700593, 'lpl_weight': 0.5391286318590843, 'ratio': 0.11221640474839122, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 1 with value: 0.835938973528496.


...

 - Metrics: Accuracy=0.9483, F1=0.9119, Recall=0.8863, Precision=0.9391
Done. Results written to cora_experimentations\cora_sar_2102151328.csv.
Average F1 over 5 seeds: 0.9065 ± 0.0029
Running experiment with seed=1:
 - alpha=0.41442746593917906, K=34, layers=2, hidden=256, out=256
 - norm=layernorm, dropout=0.13630902369716563, margin=0.5665472555924278, lpl_weight=0.6528629823821546
 - ratio=0.17130083242022817, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.9302, LPL: 1.3863, Contrastive: 0.0724
Epoch 50, Loss: 0.9216, LPL: 1.3863, Contrastive: 0.0476
Epoch 100, Loss: 0.9215, LPL: 1.3863, Contrastive: 0.0474
 - Metrics: Accuracy=0.8977, F1=0.8051, Recall=0.6993, Precision=0.9486
Running experiment with seed=2:
 - alpha=0.41442746593917906, K=34, layers=2, hidden=256, out=256
 - norm=layernorm, dropout=0.13630902369716563, margin=0.5665472555924278, lpl_weight=0.6528629823821546
 - ratio=0.17130083242022817, pos_weight=1, aggregation=mean, treatment=removal
Epoch 

[I 2025-02-21 15:15:22,163] Trial 98 finished with value: 0.8046443918453802 and parameters: {'alpha': 0.41442746593917906, 'K': 34, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'norm': 'layernorm', 'dropout': 0.13630902369716563, 'margin': 0.5665472555924278, 'lpl_weight': 0.6528629823821546, 'ratio': 0.17130083242022817, 'aggregation': 'mean', 'treatment': 'removal'}. Best is trial 94 with value: 0.9072679583787939.


 - Metrics: Accuracy=0.8962, F1=0.8031, Recall=0.7005, Precision=0.9409
Done. Results written to cora_experimentations\cora_sar_2102151404.csv.
Average F1 over 5 seeds: 0.8046 ± 0.0102
Running experiment with seed=1:
 - alpha=0.3615782966051645, K=35, layers=2, hidden=256, out=256
 - norm=layernorm, dropout=0.14697434142383314, margin=0.5821097014494977, lpl_weight=0.6433825926494596
 - ratio=0.19625485495073602, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.9362, LPL: 1.3863, Contrastive: 0.1243
 - Metrics: Accuracy=0.9472, F1=0.9113, Recall=0.8985, Precision=0.9245
Running experiment with seed=2:
 - alpha=0.3615782966051645, K=35, layers=2, hidden=256, out=256
 - norm=layernorm, dropout=0.14697434142383314, margin=0.5821097014494977, lpl_weight=0.6433825926494596
 - ratio=0.19625485495073602, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.9362, LPL: 1.3863, Contrastive: 0.1243
 - Metrics: Accuracy=0.9435, F1=0.9063, Recall=0.9046, Precision=0.90

[I 2025-02-21 15:15:57,888] Trial 99 finished with value: 0.9093667648493238 and parameters: {'alpha': 0.3615782966051645, 'K': 35, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'norm': 'layernorm', 'dropout': 0.14697434142383314, 'margin': 0.5821097014494977, 'lpl_weight': 0.6433825926494596, 'ratio': 0.19625485495073602, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 99 with value: 0.9093667648493238.


 - Metrics: Accuracy=0.9468, F1=0.9112, Recall=0.9034, Precision=0.9192
Done. Results written to cora_experimentations\cora_sar_2102151522.csv.
Average F1 over 5 seeds: 0.9094 ± 0.0022
Best trial:
  Average F1: 0.9093667648493238
  Best parameters:
    alpha: 0.3615782966051645
    K: 35
    layers: 2
    hidden_channels: 256
    out_channels: 256
    norm: layernorm
    dropout: 0.14697434142383314
    margin: 0.5821097014494977
    lpl_weight: 0.6433825926494596
    ratio: 0.19625485495073602
    aggregation: sum
    treatment: removal


### Hyperparameter Optimization Pubmed
#### SCAR 

In [9]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 15:15:57,917] A new study created in memory with name: no-name-1280e0bc-d9bb-485b-b582-15d7ebb729d4
Processing...
Done!


Running experiment with seed=1:
 - alpha=0.7796640274059625, K=29, layers=1, hidden=128, out=128
 - norm=graphnorm, dropout=0.1797844387097658, margin=0.14898368999957545, lpl_weight=0.4557055139425369
 - ratio=0.30658989044761664, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.7410, LPL: 1.3863, Contrastive: 0.2008
Epoch 50, Loss: 0.7326, LPL: 1.3863, Contrastive: 0.1852
 - Metrics: Accuracy=0.8933, F1=0.8719, Recall=0.9084, Precision=0.8381
Running experiment with seed=2:
 - alpha=0.7796640274059625, K=29, layers=1, hidden=128, out=128
 - norm=graphnorm, dropout=0.1797844387097658, margin=0.14898368999957545, lpl_weight=0.4557055139425369
 - ratio=0.30658989044761664, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.7410, LPL: 1.3863, Contrastive: 0.2008
Epoch 50, Loss: 0.7326, LPL: 1.3863, Contrastive: 0.1852
 - Metrics: Accuracy=0.8937, F1=0.8719, Recall=0.9055, Precision=0.8406
Running experiment with seed=3:
 - alpha=0.7796640274059625, K=29, l

[I 2025-02-21 15:18:30,793] Trial 0 finished with value: 0.8702607610910984 and parameters: {'alpha': 0.7796640274059625, 'K': 29, 'layers': 1, 'hidden_channels': 128, 'out_channels': 128, 'norm': 'graphnorm', 'dropout': 0.1797844387097658, 'margin': 0.14898368999957545, 'lpl_weight': 0.4557055139425369, 'ratio': 0.30658989044761664, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 0 with value: 0.8702607610910984.


 - Metrics: Accuracy=0.8926, F1=0.8708, Recall=0.9058, Precision=0.8384
Done. Results written to pubmed_experimentations\pubmed_scar_2102151557.csv.
Average F1 over 5 seeds: 0.8703 ± 0.0016
Running experiment with seed=1:
 - alpha=0.47509853891433396, K=30, layers=1, hidden=256, out=128
 - norm=None, dropout=0.33926257429148177, margin=0.11017042593138295, lpl_weight=0.2910897648260409
 - ratio=0.4156138078404573, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.6434, LPL: 1.3863, Contrastive: 0.3384
 - Metrics: Accuracy=0.8760, F1=0.8543, Recall=0.9101, Precision=0.8050
Running experiment with seed=2:
 - alpha=0.47509853891433396, K=30, layers=1, hidden=256, out=128
 - norm=None, dropout=0.33926257429148177, margin=0.11017042593138295, lpl_weight=0.2910897648260409
 - ratio=0.4156138078404573, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.6434, LPL: 1.3863, Contrastive: 0.3384
 - Metrics: Accuracy=0.8747, F1=0.8537, Recall=0.9149, Precision=0.800

[I 2025-02-21 15:20:31,377] Trial 1 finished with value: 0.8519884062003111 and parameters: {'alpha': 0.47509853891433396, 'K': 30, 'layers': 1, 'hidden_channels': 256, 'out_channels': 128, 'norm': None, 'dropout': 0.33926257429148177, 'margin': 0.11017042593138295, 'lpl_weight': 0.2910897648260409, 'ratio': 0.4156138078404573, 'aggregation': 'mean', 'treatment': 'removal'}. Best is trial 0 with value: 0.8702607610910984.


...

 - Metrics: Accuracy=0.8998, F1=0.8730, Recall=0.8621, Precision=0.8841
Done. Results written to pubmed_experimentations\pubmed_scar_2102172906.csv.
Average F1 over 5 seeds: 0.8763 ± 0.0025
Running experiment with seed=1:
 - alpha=0.7307245006173277, K=33, layers=1, hidden=128, out=64
 - norm=graphnorm, dropout=0.12842990465701737, margin=0.1927018969362656, lpl_weight=0.5574407572876022
 - ratio=0.21052269115132116, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.8549, LPL: 1.3863, Contrastive: 0.1857
Epoch 50, Loss: 0.8470, LPL: 1.3863, Contrastive: 0.1678
 - Metrics: Accuracy=0.9039, F1=0.8774, Recall=0.8608, Precision=0.8946
Running experiment with seed=2:
 - alpha=0.7307245006173277, K=33, layers=1, hidden=128, out=64
 - norm=graphnorm, dropout=0.12842990465701737, margin=0.1927018969362656, lpl_weight=0.5574407572876022
 - ratio=0.21052269115132116, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.8549, LPL: 1.3863, Contrastive: 0.1857
Epoch 50,

[I 2025-02-21 17:34:29,992] Trial 48 finished with value: 0.877568912846364 and parameters: {'alpha': 0.7307245006173277, 'K': 33, 'layers': 1, 'hidden_channels': 128, 'out_channels': 64, 'norm': 'graphnorm', 'dropout': 0.12842990465701737, 'margin': 0.1927018969362656, 'lpl_weight': 0.5574407572876022, 'ratio': 0.21052269115132116, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 15 with value: 0.8809428866883211.


 - Metrics: Accuracy=0.9061, F1=0.8799, Recall=0.8607, Precision=0.8999
Done. Results written to pubmed_experimentations\pubmed_scar_2102173217.csv.
Average F1 over 5 seeds: 0.8776 ± 0.0038
Running experiment with seed=1:
 - alpha=0.6447601667251329, K=35, layers=3, hidden=256, out=256
 - norm=layernorm, dropout=0.16513074229307365, margin=0.988214927936796, lpl_weight=0.25926532200483754
 - ratio=0.28564594150034084, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.3950, LPL: 1.3863, Contrastive: 0.0481
Epoch 50, Loss: 0.3595, LPL: 1.3863, Contrastive: 0.0000
 - Metrics: Accuracy=0.8493, F1=0.8158, Recall=0.8357, Precision=0.7969
Running experiment with seed=2:
 - alpha=0.6447601667251329, K=35, layers=3, hidden=256, out=256
 - norm=layernorm, dropout=0.16513074229307365, margin=0.988214927936796, lpl_weight=0.25926532200483754
 - ratio=0.28564594150034084, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.3950, LPL: 1.3863, Contrastive: 0.0481
Epoch 5

[I 2025-02-21 17:37:40,373] Trial 49 finished with value: 0.8178557728124629 and parameters: {'alpha': 0.6447601667251329, 'K': 35, 'layers': 3, 'hidden_channels': 256, 'out_channels': 256, 'norm': 'layernorm', 'dropout': 0.16513074229307365, 'margin': 0.988214927936796, 'lpl_weight': 0.25926532200483754, 'ratio': 0.28564594150034084, 'aggregation': 'sum', 'treatment': 'removal'}. Best is trial 15 with value: 0.8809428866883211.


 - Metrics: Accuracy=0.8502, F1=0.8174, Recall=0.8392, Precision=0.7966
Done. Results written to pubmed_experimentations\pubmed_scar_2102173430.csv.
Average F1 over 5 seeds: 0.8179 ± 0.0024
Best trial:
  Average F1: 0.8809428866883211
  Best parameters:
    alpha: 0.5311031346531587
    K: 33
    layers: 2
    hidden_channels: 128
    out_channels: 256
    norm: graphnorm
    dropout: 0.24538111087672715
    margin: 0.41638371321098333
    lpl_weight: 0.7621846841147637
    ratio: 0.23594715913885006
    aggregation: sum
    treatment: removal


### Hyperparameter Optimization Pubmed
#### SAR 

In [10]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 17:37:40,402] A new study created in memory with name: no-name-844003c5-4ac3-4390-9eab-9c8130d0dffc


Running experiment with seed=1:
 - alpha=0.8470687016196532, K=30, layers=2, hidden=256, out=64
 - norm=layernorm, dropout=0.47531813740286366, margin=0.5703046345692171, lpl_weight=0.35007282250426
 - ratio=0.447658056484299, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.5204, LPL: 1.3863, Contrastive: 0.0540
Epoch 50, Loss: 0.5163, LPL: 1.3863, Contrastive: 0.0476
Epoch 100, Loss: 0.5257, LPL: 1.3863, Contrastive: 0.0621
 - Metrics: Accuracy=0.8560, F1=0.8375, Recall=0.9290, Precision=0.7624
Running experiment with seed=2:
 - alpha=0.8470687016196532, K=30, layers=2, hidden=256, out=64
 - norm=layernorm, dropout=0.47531813740286366, margin=0.5703046345692171, lpl_weight=0.35007282250426
 - ratio=0.447658056484299, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.5204, LPL: 1.3863, Contrastive: 0.0540
Epoch 50, Loss: 0.5163, LPL: 1.3863, Contrastive: 0.0476
Epoch 100, Loss: 0.5237, LPL: 1.3863, Contrastive: 0.0590
 - Metrics: Accuracy=0.8414, F1=

[I 2025-02-21 17:40:27,164] Trial 0 finished with value: 0.8274415087727414 and parameters: {'alpha': 0.8470687016196532, 'K': 30, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'norm': 'layernorm', 'dropout': 0.47531813740286366, 'margin': 0.5703046345692171, 'lpl_weight': 0.35007282250426, 'ratio': 0.447658056484299, 'aggregation': 'mean', 'treatment': 'removal'}. Best is trial 0 with value: 0.8274415087727414.


 - Metrics: Accuracy=0.8508, F1=0.8298, Recall=0.9105, Precision=0.7623
Done. Results written to pubmed_experimentations\pubmed_sar_2102173740.csv.
Average F1 over 5 seeds: 0.8274 ± 0.0057
Running experiment with seed=1:
 - alpha=0.8952800923037607, K=34, layers=1, hidden=128, out=64
 - norm=graphnorm, dropout=0.40674542189187124, margin=0.7714836601982721, lpl_weight=0.9608682616897146
 - ratio=0.42813270752793064, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 1.3327, LPL: 1.3863, Contrastive: 0.0171
Epoch 50, Loss: 1.3326, LPL: 1.3863, Contrastive: 0.0136
Epoch 100, Loss: 1.3326, LPL: 1.3863, Contrastive: 0.0137
 - Metrics: Accuracy=0.7789, F1=0.7625, Recall=0.8886, Precision=0.6677
Running experiment with seed=2:
 - alpha=0.8952800923037607, K=34, layers=1, hidden=128, out=64
 - norm=graphnorm, dropout=0.40674542189187124, margin=0.7714836601982721, lpl_weight=0.9608682616897146
 - ratio=0.42813270752793064, pos_weight=1, aggregation=mean, treatment=relabeling


[I 2025-02-21 17:43:05,208] Trial 1 finished with value: 0.7832610605194236 and parameters: {'alpha': 0.8952800923037607, 'K': 34, 'layers': 1, 'hidden_channels': 128, 'out_channels': 64, 'norm': 'graphnorm', 'dropout': 0.40674542189187124, 'margin': 0.7714836601982721, 'lpl_weight': 0.9608682616897146, 'ratio': 0.42813270752793064, 'aggregation': 'mean', 'treatment': 'relabeling'}. Best is trial 0 with value: 0.8274415087727414.


...

 - Metrics: Accuracy=0.8995, F1=0.8667, Recall=0.8179, Precision=0.9217
Done. Results written to pubmed_experimentations\pubmed_sar_2102220030.csv.
Average F1 over 5 seeds: 0.8684 ± 0.0023
Running experiment with seed=1:
 - alpha=0.48256039271278767, K=33, layers=2, hidden=128, out=128
 - norm=None, dropout=0.15901028160211172, margin=0.10639351341870959, lpl_weight=0.2325810829408509
 - ratio=0.20977693064439082, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.4932, LPL: 1.3863, Contrastive: 0.2225
Epoch 50, Loss: 0.4793, LPL: 1.3863, Contrastive: 0.2044
 - Metrics: Accuracy=0.9024, F1=0.8747, Recall=0.8535, Precision=0.8971
Running experiment with seed=2:
 - alpha=0.48256039271278767, K=33, layers=2, hidden=128, out=128
 - norm=None, dropout=0.15901028160211172, margin=0.10639351341870959, lpl_weight=0.2325810829408509
 - ratio=0.20977693064439082, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.4932, LPL: 1.3863, Contrastive: 0.2225
Epoch 50

[I 2025-02-21 22:05:41,359] Trial 98 finished with value: 0.8731123342367499 and parameters: {'alpha': 0.48256039271278767, 'K': 33, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'norm': None, 'dropout': 0.15901028160211172, 'margin': 0.10639351341870959, 'lpl_weight': 0.2325810829408509, 'ratio': 0.20977693064439082, 'aggregation': 'sum', 'treatment': 'relabeling'}. Best is trial 22 with value: 0.8749015333712882.


 - Metrics: Accuracy=0.8999, F1=0.8709, Recall=0.8447, Precision=0.8987
Done. Results written to pubmed_experimentations\pubmed_sar_2102220305.csv.
Average F1 over 5 seeds: 0.8731 ± 0.0035
Running experiment with seed=1:
 - alpha=0.33737490324687214, K=31, layers=2, hidden=128, out=64
 - norm=None, dropout=0.15254646016424606, margin=0.16780192566367524, lpl_weight=0.6003564058838708
 - ratio=0.20517796411393624, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.9109, LPL: 1.3863, Contrastive: 0.1968
Epoch 50, Loss: 0.9036, LPL: 1.3863, Contrastive: 0.1785
 - Metrics: Accuracy=0.9015, F1=0.8731, Recall=0.8490, Precision=0.8987
Running experiment with seed=2:
 - alpha=0.33737490324687214, K=31, layers=2, hidden=128, out=64
 - norm=None, dropout=0.15254646016424606, margin=0.16780192566367524, lpl_weight=0.6003564058838708
 - ratio=0.20517796411393624, pos_weight=1, aggregation=sum, treatment=relabeling
Epoch 0, Loss: 0.9109, LPL: 1.3863, Contrastive: 0.1968
Epoch 50, 

[I 2025-02-21 22:08:00,464] Trial 99 finished with value: 0.8701494293856019 and parameters: {'alpha': 0.33737490324687214, 'K': 31, 'layers': 2, 'hidden_channels': 128, 'out_channels': 64, 'norm': None, 'dropout': 0.15254646016424606, 'margin': 0.16780192566367524, 'lpl_weight': 0.6003564058838708, 'ratio': 0.20517796411393624, 'aggregation': 'sum', 'treatment': 'relabeling'}. Best is trial 22 with value: 0.8749015333712882.


 - Metrics: Accuracy=0.8978, F1=0.8683, Recall=0.8436, Precision=0.8944
Done. Results written to pubmed_experimentations\pubmed_sar_2102220541.csv.
Average F1 over 5 seeds: 0.8701 ± 0.0030
Best trial:
  Average F1: 0.8749015333712882
  Best parameters:
    alpha: 0.25503455062799246
    K: 34
    layers: 2
    hidden_channels: 128
    out_channels: 128
    norm: graphnorm
    dropout: 0.34674827917164924
    margin: 0.16177445978937793
    lpl_weight: 0.8360928606739102
    ratio: 0.21468635530396302
    aggregation: sum
    treatment: relabeling


### Hyperparameter Optimization WikiCS
#### SCAR 

In [11]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-21 22:08:00,492] A new study created in memory with name: no-name-dd1a6520-6978-4618-b794-15799f53f943
Processing...
Done!


Running experiment with seed=1:
 - alpha=0.26230288428650056, K=28, layers=1, hidden=64, out=128
 - norm=graphnorm, dropout=0.15989127092269012, margin=0.1360543722031557, lpl_weight=0.7861329047415934
 - ratio=0.33320739969764335, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.1423, LPL: 1.3863, Contrastive: 0.2456
 - Metrics: Accuracy=0.9227, F1=0.8492, Recall=0.9511, Precision=0.7670
Running experiment with seed=2:
 - alpha=0.26230288428650056, K=28, layers=1, hidden=64, out=128
 - norm=graphnorm, dropout=0.15989127092269012, margin=0.1360543722031557, lpl_weight=0.7861329047415934
 - ratio=0.33320739969764335, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.1423, LPL: 1.3863, Contrastive: 0.2456
 - Metrics: Accuracy=0.9296, F1=0.8619, Recall=0.9601, Precision=0.7820
Running experiment with seed=3:
 - alpha=0.26230288428650056, K=28, layers=1, hidden=64, out=128
 - norm=graphnorm, dropout=0.15989127092269012, margin=0.1360543722031557, lpl_weig

[I 2025-02-21 22:09:43,133] Trial 0 finished with value: 0.8593218951028216 and parameters: {'alpha': 0.26230288428650056, 'K': 28, 'layers': 1, 'hidden_channels': 64, 'out_channels': 128, 'norm': 'graphnorm', 'dropout': 0.15989127092269012, 'margin': 0.1360543722031557, 'lpl_weight': 0.7861329047415934, 'ratio': 0.33320739969764335, 'aggregation': 'mean', 'treatment': 'removal'}. Best is trial 0 with value: 0.8593218951028216.


 - Metrics: Accuracy=0.9292, F1=0.8610, Recall=0.9582, Precision=0.7817
Done. Results written to wiki-cs_experimentations\wikics_sar_2102220800.csv.
Average F1 over 5 seeds: 0.8593 ± 0.0053
Running experiment with seed=1:
 - alpha=0.9732421061358966, K=30, layers=2, hidden=256, out=128
 - norm=graphnorm, dropout=0.12516915630102776, margin=0.9613217965088279, lpl_weight=0.8738782539513014
 - ratio=0.11919029146719083, pos_weight=1, aggregation=sum, treatment=relabeling


ERROR:root:Anomaly detector error: ValueError('Input contains NaN.')


...

 - Metrics: Accuracy=0.9545, F1=0.8969, Recall=0.8638, Precision=0.9327
Done. Results written to wiki-cs_experimentations\wikics_sar_2202024209.csv.
Average F1 over 5 seeds: 0.8978 ± 0.0015
Running experiment with seed=1:
 - alpha=0.3890407667243634, K=34, layers=2, hidden=256, out=256
 - norm=layernorm, dropout=0.3684329397541969, margin=0.28213984967210753, lpl_weight=0.5692192747618261
 - ratio=0.17160513007470618, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.9023, LPL: 1.3863, Contrastive: 0.2627
 - Metrics: Accuracy=0.9346, F1=0.8680, Recall=0.9392, Precision=0.8069
Running experiment with seed=2:
 - alpha=0.3890407667243634, K=34, layers=2, hidden=256, out=256
 - norm=layernorm, dropout=0.3684329397541969, margin=0.28213984967210753, lpl_weight=0.5692192747618261
 - ratio=0.17160513007470618, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.9023, LPL: 1.3863, Contrastive: 0.2627
 - Metrics: Accuracy=0.9345, F1=0.8677, Recall=0.9388, P

[I 2025-02-22 02:45:43,053] Trial 98 finished with value: 0.8690012075211317 and parameters: {'alpha': 0.3890407667243634, 'K': 34, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'norm': 'layernorm', 'dropout': 0.3684329397541969, 'margin': 0.28213984967210753, 'lpl_weight': 0.5692192747618261, 'ratio': 0.17160513007470618, 'aggregation': 'mean', 'treatment': 'relabeling'}. Best is trial 65 with value: 0.9032380231016853.


 - Metrics: Accuracy=0.9367, F1=0.8722, Recall=0.9436, Precision=0.8108
Done. Results written to wiki-cs_experimentations\wikics_sar_2202024355.csv.
Average F1 over 5 seeds: 0.8690 ± 0.0017
Running experiment with seed=1:
 - alpha=0.2674220264381494, K=35, layers=2, hidden=256, out=256
 - norm=layernorm, dropout=0.29383389263041026, margin=0.3220483530925544, lpl_weight=0.24958944411343165
 - ratio=0.1328521041800462, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.5294, LPL: 1.3863, Contrastive: 0.2444
 - Metrics: Accuracy=0.9525, F1=0.8970, Recall=0.9033, Precision=0.8907
Running experiment with seed=2:
 - alpha=0.2674220264381494, K=35, layers=2, hidden=256, out=256
 - norm=layernorm, dropout=0.29383389263041026, margin=0.3220483530925544, lpl_weight=0.24958944411343165
 - ratio=0.1328521041800462, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.5294, LPL: 1.3863, Contrastive: 0.2444
 - Metrics: Accuracy=0.9530, F1=0.8981, Recall=0.9044, P

[I 2025-02-22 02:47:28,648] Trial 99 finished with value: 0.8966641957005189 and parameters: {'alpha': 0.2674220264381494, 'K': 35, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'norm': 'layernorm', 'dropout': 0.29383389263041026, 'margin': 0.3220483530925544, 'lpl_weight': 0.24958944411343165, 'ratio': 0.1328521041800462, 'aggregation': 'mean', 'treatment': 'relabeling'}. Best is trial 65 with value: 0.9032380231016853.


 - Metrics: Accuracy=0.9525, F1=0.8970, Recall=0.9033, Precision=0.8907
Done. Results written to wiki-cs_experimentations\wikics_sar_2202024543.csv.
Average F1 over 5 seeds: 0.8967 ± 0.0010
Best trial:
  Average F1: 0.9032380231016853
  Best parameters:
    alpha: 0.24420575134921743
    K: 35
    layers: 2
    hidden_channels: 256
    out_channels: 256
    norm: layernorm
    dropout: 0.3599991944711314
    margin: 0.103458174066901
    lpl_weight: 0.3376161192895486
    ratio: 0.12172103712818252
    aggregation: mean
    treatment: relabeling


### Hyperparameter Optimization WikiCS
#### SAR 

In [12]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,#trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-22 02:47:28,675] A new study created in memory with name: no-name-f19039dc-b2d8-4648-8747-7fb17d61d8be


Running experiment with seed=1:
 - alpha=0.3550773461077843, K=28, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.17407591637864495, margin=0.8374569479625883, lpl_weight=0.8173330453434202
 - ratio=0.3813444753483398, pos_weight=1, aggregation=sum, treatment=removal


ERROR:root:Anomaly detector error: ValueError('Input contains NaN.')


Epoch 0, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 50, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 100, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 150, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 200, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 250, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 300, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 350, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 400, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 450, Loss: nan, LPL: 1.3863, Contrastive: nan
 - Metrics: Accuracy=0.8799, F1=0.7801, Recall=0.9302, Precision=0.6717
Running experiment with seed=2:
 - alpha=0.3550773461077843, K=28, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.17407591637864495, margin=0.8374569479625883, lpl_weight=0.8173330453434202
 - ratio=0.3813444753483398, pos_weight=1, aggregation=sum, treatment=removal


ERROR:root:Anomaly detector error: ValueError('Input contains NaN.')


...

 - Metrics: Accuracy=0.9450, F1=0.8864, Recall=0.9365, Precision=0.8414
Done. Results written to wiki-cs_experimentations\wikics_sar_2202035308.csv.
Average F1 over 5 seeds: 0.8825 ± 0.0031
Running experiment with seed=1:
 - alpha=0.5186537938015142, K=34, layers=2, hidden=64, out=256
 - norm=graphnorm, dropout=0.45413115096218115, margin=0.14513996325002754, lpl_weight=0.4768964848328673
 - ratio=0.13048294540682323, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.8083, LPL: 1.3863, Contrastive: 0.2814
 - Metrics: Accuracy=0.9507, F1=0.8926, Recall=0.8947, Precision=0.8904
Running experiment with seed=2:
 - alpha=0.5186537938015142, K=34, layers=2, hidden=64, out=256
 - norm=graphnorm, dropout=0.45413115096218115, margin=0.14513996325002754, lpl_weight=0.4768964848328673
 - ratio=0.13048294540682323, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.8083, LPL: 1.3863, Contrastive: 0.2814
 - Metrics: Accuracy=0.9505, F1=0.8922, Recall=0.8944, P

[I 2025-02-22 03:57:24,656] Trial 28 finished with value: 0.8930925339787749 and parameters: {'alpha': 0.5186537938015142, 'K': 34, 'layers': 2, 'hidden_channels': 64, 'out_channels': 256, 'norm': 'graphnorm', 'dropout': 0.45413115096218115, 'margin': 0.14513996325002754, 'lpl_weight': 0.4768964848328673, 'ratio': 0.13048294540682323, 'aggregation': 'mean', 'treatment': 'relabeling'}. Best is trial 9 with value: 0.8965259618976467.


 - Metrics: Accuracy=0.9512, F1=0.8937, Recall=0.8959, Precision=0.8915
Done. Results written to wiki-cs_experimentations\wikics_sar_2202035540.csv.
Average F1 over 5 seeds: 0.8931 ± 0.0016
Running experiment with seed=1:
 - alpha=0.3288732664619557, K=34, layers=2, hidden=64, out=64
 - norm=graphnorm, dropout=0.40592581280698875, margin=0.16108055330874504, lpl_weight=0.4642332714768575
 - ratio=0.24161947437424713, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.7837, LPL: 1.3863, Contrastive: 0.2615
Epoch 50, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 100, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 150, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 200, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 250, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 300, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 350, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 400, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 450, Loss: nan, LPL: 1.3863, Contrastive: nan


ERROR:root:Anomaly detector WeightedIsoForest error: ValueError('Input contains NaN.')
[W 2025-02-22 03:58:03,577] Trial 29 failed with parameters: {'alpha': 0.3288732664619557, 'K': 34, 'layers': 2, 'hidden_channels': 64, 'out_channels': 64, 'norm': 'graphnorm', 'dropout': 0.40592581280698875, 'margin': 0.16108055330874504, 'lpl_weight': 0.4642332714768575, 'ratio': 0.24161947437424713, 'aggregation': 'sum', 'treatment': 'removal'} because of the following error: NotFittedError('need to call fit or load_model beforehand').
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_220368\2657924698.py", line 34, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNI

NotFittedError: need to call fit or load_model beforehand